<img src=https://i.ytimg.com/vi/knxlmCVFAZI/maxresdefault.jpg>

In [5]:
# Instalar el módulo Twilio usando pip3
#!pip3 install twilio

# Instalar el módulo BeautifulSoup usando pip
#!pip install BeautifulSoup

# Instalar el módulo tqdm usando pip
#!pip install tqdm

# Instalar el módulo beautifulsoup4 usando pip
#!pip install beautifulsoup4

In [6]:
# Importar módulos necesarios

import os  # Para trabajar con funcionalidades del sistema operativo
from twilio.rest import Client  # Para enviar mensajes de texto y llamadas usando Twilio
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI  # Importar configuraciones de Twilio
import time  # Para trabajar con funciones relacionadas al tiempo

from requests import Request, Session  # Para hacer solicitudes HTTP
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects  # Importar excepciones relacionadas con solicitudes HTTP
import json  # Para trabajar con formato JSON

import pandas as pd  # Para el manejo y análisis de datos en forma de tablas
import requests  # Para realizar solicitudes HTTP
from bs4 import BeautifulSoup  # Para hacer análisis de documentos HTML
from tqdm import tqdm  # Para mostrar barras de progreso en bucles

from datetime import datetime  # Para trabajar con fechas y horas


# Armado de la URL

In [7]:
query = 'Cucuta'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=989746c555d44e2caf005936232908&q=Cucuta&days=1&aqi=no&alerts=no'

In [8]:
response = requests.get(url_clima).json()

In [9]:
response

{'location': {'name': 'Cucuta',
  'region': 'Norte de Santander',
  'country': 'Colombia',
  'lat': 7.88,
  'lon': -72.51,
  'tz_id': 'America/Bogota',
  'localtime_epoch': 1693526755,
  'localtime': '2023-08-31 19:05'},
 'current': {'last_updated_epoch': 1693526400,
  'last_updated': '2023-08-31 19:00',
  'temp_c': 32.0,
  'temp_f': 89.6,
  'is_day': 0,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png',
   'code': 1003},
  'wind_mph': 16.1,
  'wind_kph': 25.9,
  'wind_degree': 180,
  'wind_dir': 'S',
  'pressure_mb': 1010.0,
  'pressure_in': 29.83,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 43,
  'cloud': 50,
  'feelslike_c': 37.4,
  'feelslike_f': 99.3,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 1.0,
  'gust_mph': 14.1,
  'gust_kph': 22.7},
 'forecast': {'forecastday': [{'date': '2023-08-31',
    'date_epoch': 1693440000,
    'day': {'maxtemp_c': 36.7,
     'maxtemp_f': 98.1,
     'mintemp_c': 19.5,
     'mintemp_f': 

In [10]:
# Obtener las claves disponibles en el objeto 'response'
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [11]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [12]:
len(response['forecast']['forecastday'][0]['hour']) #Horas disponibles para la consulta

24

In [13]:
#Obtenemos la fecha
response['forecast']['forecastday'][0]['date']

'2023-08-31'

In [14]:
#Obtenemos la hora
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [15]:
#condition del clima
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Clear'

In [16]:
#Temperatura en grados centigrados
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

20.7

In [17]:
#Llueve (1) o No Llueve (0)
response['forecast']['forecastday'][0]['hour'][1]['will_it_rain']

0

In [18]:
#Probabilidad de LLuvia Sí (1) , No (0)
response['forecast']['forecastday'][0]['hour'][1]['chance_of_rain']

0

# Dataframe

In [19]:
#Obtener fecha, hora, condicion del clima, temperatura,lluvia, probabilidad de lluvia
def get_forecast(response,i): #respuesta e iterador
    
    Fecha = response['forecast']['forecastday'][0]["hour"][i]['time'].split()[0]
    Hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    Condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    Temperatura = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    Lluvia = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    Probabilidad_Lluvia = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    
    return Fecha, Hora, Condicion, Temperatura, Lluvia, Probabilidad_Lluvia

In [20]:
#La libreria tqdm pérmite visualizar una barra de carga
datos = []
Horas_Disponibles = len(response['forecast']['forecastday'][0]['hour'])
for i in tqdm(range(Horas_Disponibles),colour = 'green'):
    
    datos.append(get_forecast(response,i))
    

100%|██████████| 24/24 [00:00<00:00, 12032.43it/s]


In [21]:
datos[0] #revisemos el primer registro

('2023-08-31', 0, 'Clear', 20.7, 0, 0)

In [22]:
#Creamos un dataframe con los datos obtenidos
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2023-08-31,0,Clear,20.7,0,0
1,2023-08-31,1,Clear,20.3,0,0
2,2023-08-31,2,Patchy rain possible,19.8,1,83
3,2023-08-31,3,Clear,19.7,0,0
4,2023-08-31,4,Patchy rain possible,19.6,1,76
5,2023-08-31,5,Clear,19.5,0,0
6,2023-08-31,6,Sunny,19.5,0,0
7,2023-08-31,7,Sunny,24.8,0,0
8,2023-08-31,8,Sunny,29.7,0,0
9,2023-08-31,9,Sunny,32.5,0,0


In [23]:
#filtramos los datos para Horas mayores a 6am y menores a las 22, 
# que es cuando las personas duermen 
df_rain = df[(df["Hora"]>6) & (df["Hora"] < 22)]

#Filtramos el dataframe, dejando solo Hora y Condicion 
df_rain = df_rain[["Hora", "Condicion", "Temperatura"]]
df_rain.set_index("Hora", inplace = True)


In [24]:
df_rain

,Condicion,Temperatura
Hora,,
7,Sunny,24.8
8,Sunny,29.7
9,Sunny,32.5
10,Sunny,34.4
11,Sunny,35.7
12,Sunny,36.4
13,Sunny,36.7
14,Sunny,36.7
15,Sunny,36.1


# Mensaje que se Envíara

In [25]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain) 

'\nHola! \n\n\n El pronostico del tiempo hoy 2023-08-31 en Cucuta es : \n\n\n                  Condicion  Temperatura\nHora                                   \n7                    Sunny         24.8\n8                    Sunny         29.7\n9                    Sunny         32.5\n10                   Sunny         34.4\n11                   Sunny         35.7\n12                   Sunny         36.4\n13                   Sunny         36.7\n14                   Sunny         36.7\n15                   Sunny         36.1\n16    Patchy rain possible         34.8\n17                   Sunny         33.0\n18                   Clear         26.3\n19           Partly cloudy         32.0\n20                   Clear         22.7\n21                   Clear         21.6'

In [34]:
PHONE_NUMBER

'+1 815 869 9136'

# Mensaje SMS desde Twilio

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID 
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+573005424395'
                 )

print('Mensaje Enviado ' + message.sid)